In [ ]:
import os
import sys
import django
import pandas as pd

from IPython.display import display
from pathlib import Path

CWD = Path(".").resolve()
DBF_APP_PATH = Path(CWD.parent.parent, "dbfapp")

#It is necessary to run django setup to know desulf package from app
#Is it necessary to add the path to the app to the environment
#variable path, to it possible run django know app module dbfapp.
sys.path += [str(DBF_APP_PATH)]

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dbfapp.settings")
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "True")
django.setup()

from dotenv import load_dotenv
from pathlib import Path

ENV_FILE_PATH = Path(CWD.parent.parent, "config.env")

load_dotenv(dotenv_path=ENV_FILE_PATH)

from modelbackend.management.commands.runmodels import get_signal_group_data
from modelbackend.models import NeuralPredictionResult
from modelbackend.management.commands.evaluatemodel import METRICS_MAP
from dbfcore.dataset.hooks import get_datasources_configured_with_env
from dbfcore.predictionmodel.inference import get_minmax_data_definition

In [ ]:
start = pd.Timestamp("2023")
end = pd.Timestamp("2025")
model_name = "sibf3"

In [ ]:
prediction_results = NeuralPredictionResult.objects.filter(
    deployment_name=model_name, prediction_start__range=[start, end]
)
calc_times = prediction_results.values_list("prediction_start", flat=True)

data_definitions = []

for metric in METRICS_MAP.values():
    data_definitions.append(metric.get_data_definition(calc_times))

data_definition = get_minmax_data_definition(data_definitions)
datasources = get_datasources_configured_with_env()
signal_group_data = get_signal_group_data(data_definition, datasources)

In [ ]:
from modelbackend.models import NeuralBlastFurnaceModelDeployment


deployment = NeuralBlastFurnaceModelDeployment.objects.get(name=model_name)
# model = "nacitaj model podla deploymentu pouzitim cesty"


# evaluations = {}

# for calc_time in calc_times:
#     for metric in METRICS_MAP.values():
#         result = metric.calculate_metric(calc_time, model, results, signal_group_data)
#         evaluations["nazov"] = result

# pd.Dataframe.from_dict(evaluations)

In [ ]:
from dbfcore.predictionmodel.model import PredictionModel
from dbfcore.utils import load_lightning_model_from_path_cached


model = load_lightning_model_from_path_cached(PredictionModel, "C:/Data/BF/localdeployment/si_bf3.ckpt").cpu().eval()

In [ ]:
data_df = target_data["bf3_hotmetal_chem"]
real_value = get_closest_result(data_df, calc_times + pd.Timedelta("30min"), pd.Timedelta("5min"))

if not real_value:
    return float("nan")

value_time, value = real_value
offset = value_time - calc_times

model_value = results.get_expected_value(model, offset)  # type: ignore

return abs(value - model_value)

In [ ]:
calc_times_plus = pd.DataFrame(calc_times, columns=["calc_time"]) + pd.Timedelta("30min")
calc_times_df = calc_times_plus.set_index("calc_time").sort_index()

In [ ]:
data_df = signal_group_data["bf3_hotmetal_chem"]

In [ ]:
import pandas as pd

In [ ]:
real_values = pd.merge_asof(calc_times_df, data_df, right_index=True, left_index=True, tolerance=pd.Timedelta("5min"), direction="nearest")["bf3_hotmetalsi_chem_pct"]

In [ ]:
prediction_results[0]

In [ ]:
from dbfcore.predictionmodel.inference import NeuralBlastFurnaceModelResult
import torch

results = [NeuralBlastFurnaceModelResult.from_bytes(prediction_result.encoded, prediction_result.prediction_start, model) for prediction_result in prediction_results]

In [ ]:
representations = torch.cat([res.representation for res in results])

In [ ]:
model.get_expected_values(representations, 30.0 * 60.0, 51)

In [ ]:
model.get_expected_values(representations, 30.0 * 60.0, 101)

In [ ]:
eval_time = torch.Tensor([30.0 * 60.0]).repeat(representations.size(0)).reshape(-1, 1)

In [ ]:
grid = torch.linspace(0, 1, 201)

In [ ]:
eval_value = grid.reshape(-1, 1).repeat(len(results), 1)

In [ ]:
cdf = model.forward_from_representation(representations, eval_time, eval_value)

In [ ]:
cdf.reshape(len(results), 201)

In [ ]:
from dbfcore.predictionmodel.model import pdf_to_expected_value, cdf_to_pdf, get_pdf_values


model.value_denormalizer(pdf_to_expected_value(cdf_to_pdf(cdf.reshape(len(results), 201)), get_pdf_values(grid)))